# North Korea provocations database

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_grid as altgrid
from altair_saver import save
import numpy as np
import us
import urllib.request, json
import glob
import os
import requests
from bs4 import BeautifulSoup
import os
import time
import matplotlib.pyplot as plt
import squarify
from matplotlib.pyplot import figure

In [3]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")
figure(figsize=(8, 6), dpi=80)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [4]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Download latest version of the database from here

In [5]:
# https://beyondparallel.csis.org/database-north-korean-provocations/

### Import data

In [6]:
src = pd.read_csv("data/raw/Provocations New.csv")

In [7]:
src.columns = (
    src.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=True)
    .str.replace("(", "", regex=True)
    .str.replace(")", "", regex=True)
    .str.replace("-", "_", regex=True)
)

In [8]:
src.sort_values("date", ascending=False).head()

,date,type,event,description,resources
0,2022-10-14,Other Provocation,Airplane exercise,Around 10 North Korean military aircraft condu...,https://en.yna.co.kr/view/AEN20221014000300325
1,2022-10-14,Other Provocation,Artillery Fire,"From 1:20 a.m. KST, North Korea fired 130 roun...",https://en.yna.co.kr/view/AEN20221014011051325...
2,2022-10-14,Missile Provocation,Short-range ballistic Missile Launch,North Korea fired a short-range ballistic miss...,https://en.yna.co.kr/view/AEN20221014000552325
3,2022-10-12,Missile Provocation,Long-range Cruise Missile Launch,North Korea fired two long-range strategic cru...,https://en.yna.co.kr/view/AEN20221013000652325...
4,2022-10-09,Missile Provocation,Short-range ballistic Missile Launch,North Korea fired two short-range ballistic mi...,https://en.yna.co.kr/view/AEN20221009000351325


### Parse dates

In [9]:
src["date"] = pd.to_datetime(src["date"])
src["year"] = src["date"].dt.year
src["month"] = src["date"].dt.month
src["day"] = src["date"].dt.day
src["day"] = src["date"].dt.day_name()

In [10]:
src["type"] = src["type"].str.replace(" Provocation", "")

---

## Aggregate

#### Provocation types

In [11]:
prov_types_src = (
    src.groupby(["type", "year"])
    .agg({"date": "count"})
    .reset_index()
    .rename(columns={"date": "count"})
)

In [12]:
# prov_types["date"] = pd.to_datetime("12/31/" + prov_types["year"].astype(str))

In [13]:
dummy = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vT1tUkO4UkOeOl5_O_j5yEHplUJF1NLC1u55kCWg_1HsbTputy80dZJubTzCdHCKPmTgMS8V9KS6cmM/pub?gid=0&single=true&output=csv"
)

In [14]:
prov_types = pd.merge(prov_types_src, dummy, on=["year", "type"], how="right").fillna(0)

In [15]:
prov_types["year"] = prov_types["year"].astype(str)

In [16]:
circles = (
    alt.Chart(prov_types)
    .mark_circle(thickness=2, opacity=0.5)
    .encode(
        alt.X(
            "year:O",
            axis=alt.Axis(
                values=["1960", "1970", "1980", "1990", "2000", "2010", "2022"]
            ),
            title=" ",
        ),
        alt.Y("type:N", title=" "),
        alt.Size(
            "count:Q",
            title="Missle count",
            scale=alt.Scale(range=[0, 2000]),
            legend=None,
        ),
        alt.Color("type:N", legend=None),
    )
    .properties(
        width=850,
        height=200,
        title="",
    )
)

text = (
    alt.Chart(prov_types)
    .mark_text(dx=0, dy=0, color="black")
    .encode(
        x=alt.X("year:O", stack="zero"),
        y=alt.Y("type:N"),
        text=alt.Text("count:Q"),
    )
)

(circles)

alt.Chart(...)

In [19]:
!pip uninstall altair_saver
!y

Exception ignored in: <function _releaseLock at 0x102d855e0>
Traceback (most recent call last):
  File "/Users/stiles/.pyenv/versions/3.9.9/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


^C
Traceback (most recent call last):
  File "/Users/stiles/.pyenv/versions/3.9.9/bin/pip", line 5, in <module>
    from pip._internal.cli.main import main
  File "/Users/stiles/.pyenv/versions/3.9.9/lib/python3.9/site-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/Users/stiles/.pyenv/versions/3.9.9/lib/python3.9/site-packages/pip/_internal/cli/autocompletion.py", line 12, in <module>
    from pip._internal.metadata import get_default_environment
  File "/Users/stiles/.pyenv/versions/3.9.9/lib/python3.9/site-packages/pip/_internal/metadata/__init__.py", line 9, in <module>
    from .base import BaseDistribution, BaseEnvironment, FilesystemWheel, MemoryWheel, Wheel
  File "/Users/stiles/.pyenv/versions/3.9.9/lib/python3.9/site-packages/pip/_internal/metadata/base.py", line 25, in <module>
    from pip._vendor.packaging.requirements import Requirement
  File "/Users/stiles/.pyenv/versions/3.9.9/lib/python3.9